In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
import datetime

In [27]:
# Load datasets
train = pd.read_feather("./data/train_feather")
# val = pd.read_feather("./data/val_feather")
# test = pd.read_feather("./data/test_feather")
# ids = pd.read_feather("./data/test_ids")

# train = shuffle(train)
# val = shuffle(val)

In [32]:
train = train[['store_nbr', 'item_nbr', 'onpromotion', 'year', 'month', 'dayofmonth', 'dayofweek', 'unit_sales']]
print(train.dtypes)
train.head()

store_nbr      float32
item_nbr       float32
onpromotion    float32
year           float32
month          float32
dayofmonth     float32
dayofweek      float32
unit_sales     float32
dtype: object


,store_nbr,item_nbr,onpromotion,year,month,dayofmonth,dayofweek,unit_sales
0,38.0,456875.0,0.0,2013.0,8.0,13.0,1.0,2.000
1,40.0,1968666.0,0.0,2016.0,10.0,13.0,3.0,2.000
2,33.0,1109326.0,0.0,2013.0,12.0,11.0,2.0,20.000
3,45.0,1239914.0,0.0,2015.0,1.0,17.0,5.0,20.000
4,38.0,1239897.0,0.0,2014.0,4.0,20.0,6.0,7.596


In [3]:
for col in train.columns:
    train[col] = train[col].astype(np.float32)
for col in val.columns:
    val[col] = val[col].astype(np.float32)
for col in test.columns:
    test[col] = test[col].astype(np.float32)

In [31]:
train.reset_index(inplace=True)
# val.reset_index(inplace=True)

In [ ]:
train.to_feather("./data/train_feather")
# val.to_feather("./data/val_feather")
# test.to_feather("./data/test_feather")

In [8]:
# Parameters
learning_rate = 0.001
num_steps = 300
batch_size = 256
display_step = 5

# Indices to traverse dataset
train_ptr = 0
val_ptr = 0

# Network Parameters
n_hidden_1 = 128 # 1st layer number of neurons
n_hidden_2 = 128 # 2nd layer number of neurons
n_hidden_3 = 128 # 2nd layer number of neurons
num_input = 4207 # one-hot encoding of featuers
num_classes = 1 # regression

# Sizes of datasets
num_samples = train.shape[0]
num_samples_val = val.shape[0]
num_samples_test = test.shape[0]

# For one-hot encoding of the features
encoding_vals = np.load("./data/encoding_vals.npy")
enc = OneHotEncoder(dtype=np.float32)
enc.fit(encoding_vals)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float32'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [5]:
# Traverses dataset using begin/end
# One-hot encodes all the features as they are all categorical
def get_batch_train(begin, end):
    batch = train.iloc[begin:end,:].values
    labels = batch[:,-1]
    features = batch[:,:-1]
    one_hots = enc.transform(features).toarray()

    return (tf.convert_to_tensor(one_hots), tf.convert_to_tensor(labels))

def get_batch_val(begin, end):
    batch = val.iloc[begin:end,:].values
    labels = batch[:,-1]
    features = batch[:,:-1]
    one_hots = enc.transform(features).toarray()

    return (tf.convert_to_tensor(one_hots), tf.convert_to_tensor(labels))

def get_batch_test(begin, end):
    batch = test.iloc[begin:end,:].values
    one_hots = enc.transform(batch).toarray()

    return tf.convert_to_tensor(one_hots)

def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [6]:
tf.reset_default_graph()


# tf Graph input
X = tf.placeholder("float")
Y = tf.placeholder("float")

# Store layers weight & bias
# h10 = tf.get_variable('h10', [num_input, n_hidden_1], initalizer=tf.random_normal_initializer)
with tf.name_scope('weights'):
    weights = {
        'h1': tf.get_variable('h1', [num_input, n_hidden_1], tf.float32, tf.random_normal_initializer()),
        'h2': tf.get_variable('h2', [n_hidden_1, n_hidden_2], tf.float32, tf.random_normal_initializer()),
        'h3': tf.get_variable('h3', [n_hidden_2, n_hidden_3], tf.float32, tf.random_normal_initializer()),
        'out': tf.get_variable('out', [n_hidden_3, num_classes], tf.float32, tf.random_normal_initializer())  
    }
    
with tf.name_scope('biases'):
    biases = {
        'b1': tf.get_variable('b1', [n_hidden_1], tf.float32, tf.random_normal_initializer()),
        'b2': tf.get_variable('b2', [n_hidden_2], tf.float32, tf.random_normal_initializer()),
        'b3': tf.get_variable('b3', [n_hidden_3], tf.float32, tf.random_normal_initializer()),
        'outb': tf.get_variable('outb', [num_classes], tf.float32, tf.random_normal_initializer())
    }

global_step = tf.Variable(0, name='global_step', trainable=False)

# should make a loop
# variable_summaries(weights['h1'])
# variable_summaries(weights['h2'])
# variable_summaries(weights['h3'])
# variable_summaries(weights['out'])
# variable_summaries(biases['b1'])
# variable_summaries(biases['b2'])
# variable_summaries(biases['b3'])
# variable_summaries(biases['outb'])

# Create model
with tf.name_scope('nn'):
    def neural_net(x):
        # Hidden fully connected layer with 128 neurons
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        tf.summary.histogram("L1pre-activations", layer_1)
        l1_act = tf.nn.relu(layer_1)
        tf.summary.histogram("L1activated", l1_act)
        # Hidden fully connected layer with 128 neurons
        layer_2 = tf.add(tf.matmul(l1_act, weights['h2']), biases['b2'])
        tf.summary.histogram("L2pre-activations", layer_2)
        l2_act = tf.nn.relu(layer_2)
        tf.summary.histogram("L2activated", l2_act)
        # Hidden fully connected layer with 128 neurons
        layer_3 = tf.add(tf.matmul(l2_act, weights['h3']), biases['b3'])
        tf.summary.histogram("L3pre-activations", layer_3)
        l3_act = tf.nn.relu(layer_3)
        tf.summary.histogram("L3activated", l3_act)
        # Output fully connected layer with a neuron for prediction
        out_layer = tf.matmul(l3_act, weights['out']) + biases['outb']
        return out_layer

In [7]:
# Forward prop
prediction = neural_net(X)
tf.summary.histogram("output", prediction)

# Define loss and optimizer
loss_op = tf.losses.mean_squared_error(labels=Y, predictions=prediction)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op, global_step=global_step)

# Evaluation
mae = tf.metrics.mean_absolute_error(labels=Y, predictions=prediction)

# Tensorboard
tb_loss = tf.summary.scalar("loss", loss_op)
tb_mae = tf.summary.scalar("MAE", tf.reduce_mean(mae))
merged = tf.summary.merge_all()

saver = tf.train.Saver(tf.global_variables())

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

In [10]:
# Start training
with tf.Session() as sess:

    print("Restoring and Intializing")
    saver.restore(sess, "./model_checks/model.ckpt")
#     print("h1 : %s" % weights['h1'].eval())

    # Run the initializer
#     sess.run(init)
    sess.run(init_l)
#     print("h1 : %s" % weights['h1'].eval())


    # Tensorboard writers
    train_writer = tf.summary.FileWriter("./log_tb" + '/train', sess.graph)
    test_writer = tf.summary.FileWriter("./log_tb" + '/test')

    print("Begin datetime: " + str(datetime.datetime.now()))
    # TRAINING ///
    for step in range(1, num_steps+1):
        try:
            # Get batch and use begin/end indices to traverse dataset
            batch_x, batch_y = sess.run(get_batch_train(train_ptr, train_ptr+batch_size))
            train_ptr+=batch_size
            
            # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        except IndexError:
            print("Finished an epoch, shuffling dataset")
            shuffle(train)
            train_ptr=0

        # VALIDATING and printing stats
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            summary, loss, mean_abs_train = sess.run([merged, loss_op, mae], feed_dict={X: batch_x, Y: batch_y})
            train_writer.add_summary(summary, tf.train.global_step(sess, global_step))

            # Validation accuracy
            try:
                val_batch_x, val_batch_y = sess.run(get_batch_val(val_ptr, val_ptr+batch_size))
                val_ptr+=batch_size
            except StopIteration:
                print("Reshuffling validation set")
                shuffle(val)
                val_ptr=0

                val_batch_x, val_batch_y = sess.run(get_batch_val(val_ptr, val_ptr+batch_size))
                val_ptr+=batch_size

            summary, mean_abs_val = sess.run([merged, mae], feed_dict={X: val_batch_x, Y: val_batch_y})
            test_writer.add_summary(summary, tf.train.global_step(sess, global_step))

            print("Step " + str(step) + "/" + str(tf.train.global_step(sess, global_step)) + ", Minibatch Loss = " + \
                  str(loss) + ", Training MAE = " + \
                  str(mean_abs_train) + ", Eval MAE = " + str(mean_abs_val))


    print("End datetime: " + str(datetime.datetime.now()))
    # Save model
#     print("h1 : %s" % weights['h1'].eval())
    save_path = saver.save(sess, "./model_checks/model.ckpt")
    print("Model saved at: " + str(save_path))

    # Predict values for submission
#     submission = np.array([[0,0]])
#     counter = 0
#     test_ptr = 0
    
#     while(True):
#         if (test_ptr > num_samples_test):
#             np.savetxt("./submission.txt", submission, delimiter=',')
#             print("Saved Submissions!")
#             break
            
#         try:
#             if(counter % 10000 == 0):
#                 print("batch # " + str(counter))
#             counter+=1

#             test_batch = sess.run(get_batch_test(test_ptr, test_ptr+batch_size))
#             test_ids = ids.iloc[test_ptr:test_ptr+batch_size].values.reshape((batch_size,-1))
#             test_ptr+=batch_size

#             predictions = sess.run([prediction], feed_dict={X: test_batch})
#             preds = sess.run(tf.squeeze(preds)).reshape((batch_size, -1))
#             new_subs = sess.run(tf.squeeze(np.array([test_ids, preds]).T))
#             submission = np.concatenate((submission, new_subs))
#             print("Concatenated!")

#         except IndexError:
#             np.savetxt("./submission.txt", submission, delimiter=',')
#             print("Saved Submissions!")
#             break

    print("Optimization Finished!")

Restoring and Intializing
INFO:tensorflow:Restoring parameters from ./model_checks/model.ckpt
Begin datetime: 2018-01-09 05:21:27.931862
Step 1/301, Minibatch Loss = 31872.0, Training MAE = (0.0, 138.83485), Eval MAE = (138.83485, 145.81924)
Step 5/305, Minibatch Loss = 31621.5, Training MAE = (145.81924, 144.66667), Eval MAE = (144.66667, 146.38008)
Step 10/310, Minibatch Loss = 29482.9, Training MAE = (146.38008, 145.15881), Eval MAE = (145.15881, 148.35052)
Step 15/315, Minibatch Loss = 29249.6, Training MAE = (148.35052, 146.76233), Eval MAE = (146.76233, 146.7634)
Step 20/320, Minibatch Loss = 30999.7, Training MAE = (146.7634, 146.08112), Eval MAE = (146.08112, 147.23471)
Step 25/325, Minibatch Loss = 24157.6, Training MAE = (147.23471, 145.25014), Eval MAE = (145.25014, 145.9659)
Step 30/330, Minibatch Loss = 26349.6, Training MAE = (145.9659, 144.1429), Eval MAE = (144.1429, 144.09691)
Step 35/335, Minibatch Loss = 23184.8, Training MAE = (144.09691, 142.53951), Eval MAE = (142

ValueError: GraphDef cannot be larger than 2GB.